In [16]:
from keras.datasets import fashion_mnist
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, Lambda, MaxPooling2D, BatchNormalization
from tensorflow.keras.models import Sequential,load_model

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "4"

In [24]:
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()
print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)
x_train = tf.expand_dims(x_train, axis = -1)
x_train = tf.keras.backend.repeat_elements(x=x_train, rep=3, axis=-1)
x_test = tf.expand_dims(x_test, axis = -1)
x_test = tf.keras.backend.repeat_elements(x=x_test, rep=3, axis=-1)

(60000, 28, 28) (60000,) (10000, 28, 28) (10000,)


In [25]:
num_classes = 10
# Convert class vectors to binary class matrices.
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)
print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)

(60000, 28, 28, 3) (60000, 10) (10000, 28, 28, 3) (10000, 10)


In [52]:
input_shape = (28,28,3)

model = Sequential()
# Block 1
model.add(Conv2D(64, (3, 3), padding='same', name = 'block1_conv1', 
                 input_shape=input_shape))
# model.add(BatchNormalization(name = 'block1_bn1'))
model.add(Activation('relu', name = 'block1_act1'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), name='block1_pool1'))

model.add(Conv2D(128, (3, 3), padding='same', name = 'block1_conv2'))
# model.add(BatchNormalization(name = 'block1_bn2'))
model.add(Activation('relu', name = 'block1_act2'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), name='block1_pool2'))

model.add(Conv2D(256, (3, 3), padding='same', name = 'block1_conv3'))
# model.add(BatchNormalization(name = 'block1_bn3'))
model.add(Activation('relu', name = 'block1_act3'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), name='block1_pool3'))

# model.add(Conv2D(256, (3, 3), padding='same', name = 'block1_conv4'))
# # model.add(BatchNormalization(name = 'block1_bn4'))
# model.add(Activation('relu', name = 'block1_act4'))
# model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), name='block1_pool4'))

# top
model.add(Flatten(name='flatten'))
model.add(Dense(1024, activation='relu', name='fc1'))
model.add(Dropout(0.5, name='drop1'))
model.add(Dense(32, activation='relu', name='fc2'))
model.add(Dropout(0.5, name='drop2'))
# model.add(Dense(64, activation='relu', name='fc3'))
# model.add(Dropout(0.5, name='drop3'))
model.add(Dense(num_classes, activation='softmax', name='predictions'))

In [53]:
model.summary()

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 28, 28, 64)        1792      
_________________________________________________________________
block1_act1 (Activation)     (None, 28, 28, 64)        0         
_________________________________________________________________
block1_pool1 (MaxPooling2D)  (None, 14, 14, 64)        0         
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 14, 14, 128)       73856     
_________________________________________________________________
block1_act2 (Activation)     (None, 14, 14, 128)       0         
_________________________________________________________________
block1_pool2 (MaxPooling2D)  (None, 7, 7, 128)         0         
_________________________________________________________________
block1_conv3 (Conv2D)        (None, 7, 7, 256)       

In [54]:
# model = load_model("./fmnist_CNN.h5")
opt = tf.keras.optimizers.Adam(learning_rate=0.00005)
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath="./fmnist_CNN.h5", 
                                                monitor='val_accuracy', 
                                                verbose=1, 
                                                save_best_only=True, mode = 'max')
callbacks_list = [checkpoint]
model.fit(x_train, y_train, batch_size=256, epochs=20, shuffle=True,
          validation_data=(x_test[0:5000], y_test[0:5000]), callbacks=callbacks_list)

Epoch 1/20
235/235 [==============================] - 2s 6ms/step - loss: 2.7924 - accuracy: 0.1684 - val_loss: 1.8275 - val_accuracy: 0.3320

Epoch 00001: val_accuracy improved from -inf to 0.33200, saving model to ./fmnist_CNN.h5
Epoch 2/20
235/235 [==============================] - 1s 5ms/step - loss: 1.8196 - accuracy: 0.3451 - val_loss: 1.2573 - val_accuracy: 0.5606

Epoch 00002: val_accuracy improved from 0.33200 to 0.56060, saving model to ./fmnist_CNN.h5
Epoch 3/20
235/235 [==============================] - 1s 6ms/step - loss: 1.3510 - accuracy: 0.5056 - val_loss: 0.6733 - val_accuracy: 0.7988

Epoch 00003: val_accuracy improved from 0.56060 to 0.79880, saving model to ./fmnist_CNN.h5
Epoch 4/20
235/235 [==============================] - 1s 5ms/step - loss: 0.9515 - accuracy: 0.6675 - val_loss: 0.5201 - val_accuracy: 0.8398

Epoch 00004: val_accuracy improved from 0.79880 to 0.83980, saving model to ./fmnist_CNN.h5
Epoch 5/20
235/235 [==============================] - 1s 5ms/st

In [55]:
model.evaluate(x=x_test, y=y_test, batch_size=None, 
               verbose=1)

313/313 [==============================] - 1s 2ms/step - loss: 0.2796 - accuracy: 0.9079


[0.27957823872566223, 0.9078999757766724]